In [1]:
# Install a pip package in the current Jupyter kernel
import mariadb as mdb
import json
import pandas as pd
import glob
import os
from math import ceil
import pleiades as ple

In [2]:
server_config = {
    'user': 'root',
    # 'password': '1234',
    'host': '127.0.0.1',
    'port': 3306,
    # 'database': 'testDB',
    # Unknown how to set local_infile.
    # 'local_infile': True
}

In [3]:
#creating coursor object
db = mdb.connect(**server_config)
cursor = db.cursor()

In [4]:
dbname = 'testDB'
command = f'USE {dbname};'
try:
    cursor.execute(command)
except mdb.ProgrammingError:
    print('database does not exist')

In [13]:
df = pd.read_csv('../data/konosuba.csv')
df

,id,name,class,race,male,age
0,1,Kazuma,adventurer,human,True,17
1,2,Aqua,priest,god,False,999
2,3,Megumin,crusader,human,False,14
3,4,Darkness,wizard,human,False,19
4,5,Chris,thief,human,False,15
5,6,Yunyun,wizard,human,False,14
6,7,Wiz,wizard,lich,False,20
7,8,Kyouya,swordmaster,human,True,20


In [5]:
cz = ple.CZ(cursor)

In [8]:
print(cz.table_from_csv('../data/konosuba.csv', 'id', print_only=True))

CREATE TABLE konosuba(
id INT,
name VARCHAR(50),
class VARCHAR(50),
race VARCHAR(50),
male BOOLEAN,
age INT,
PRIMARY KEY (id)
);


In [9]:
cz.table_from_csv('../data/konosuba.csv', 'id')

'table konosuba created.'

In [10]:
cursor.execute('DESC konosuba')

In [11]:
cursor.fetchall()

[('id', 'int(11)', 'NO', 'PRI', None, ''),
 ('name', 'varchar(50)', 'YES', '', None, ''),
 ('class', 'varchar(50)', 'YES', '', None, ''),
 ('race', 'varchar(50)', 'YES', '', None, ''),
 ('male', 'tinyint(1)', 'YES', '', None, ''),
 ('age', 'int(11)', 'YES', '', None, '')]

In [78]:
cursor.execute("""CREATE TABLE passengers(
id SERIAL PRIMARY KEY,
sex BOOLEAN
);""")

In [79]:
cursor.execute("""INSERT INTO passengers (ID,SEX)
VALUES (1, 'True');
""")

In [80]:
cursor.execute("""SELECT id,sex
FROM passengers
""")

In [81]:
cursor.fetchall()

[(1, 0)]

In [64]:
command = """LOAD DATA LOCAL INFILE 'source.csv' INTO target_db.target_table FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED BY '\r\n'"""

In [58]:
command = command + repr("\r\n'")

In [65]:
print(command)

LOAD DATA LOCAL INFILE 'source.csv' INTO target_db.target_table FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED BY '
'


In [66]:
cursor.execute(command)

ProgrammingError: You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'target_db.target_table FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED' at line 1

In [77]:
cursor.execute("""DROP TABLE passengers""")

In [6]:
cursor.execute("""CREATE TABLE passengers(
id SERIAL PRIMARY KEY,
name text,
sex text,
age float,
sibsp integer,
parch integer)""")

In [7]:
cursor.execute("""DROP TABLE passengers""")

In [4]:
#creating tables 
cursor.execute("""CREATE TABLE passengers(
id SERIAL PRIMARY KEY,
name text,
sex text,
age float,
sibsp integer,
parch integer)""")

cursor.execute("""CREATE TABLE survival(
id SERIAL PRIMARY KEY,
survived integer)""")

cursor.execute("""CREATE TABLE tripInfo(
id SERIAL PRIMARY KEY,
pclass integer,
ticket text,
fare float,
cabin text,
embarked text)""")

connection.commit()

In [5]:
#Quering Information Shema table
sql = """
SELECT "table_name","column_name", "data_type", "table_schema"
FROM INFORMATION_SCHEMA.COLUMNS
WHERE "table_schema" = 'public'
ORDER BY table_name  
"""
pd.read_sql(sql, con=connection)

,table_name,column_name,data_type,table_schema
0,passengers,name,text,public
1,passengers,sex,text,public
2,passengers,age,double precision,public
3,passengers,id,integer,public
4,passengers,parch,integer,public
5,passengers,sibsp,integer,public
6,survival,id,integer,public
7,survival,survived,integer,public
8,tripinfo,fare,double precision,public
9,tripinfo,pclass,integer,public


In [6]:
# Copy data from csv files to tables:

with open('passengers.csv', 'r') as row:
    next(row)# Skip the header row.
    cursor.copy_from(row, 'passengers', sep=',')
    
with open('survival.csv', 'r') as row:
    next(row) 
    cursor.copy_from(row, 'survival', sep=',')

with open('trip_info.csv', 'r') as row:
    next(row)
    cursor.copy_from(row, 'tripInfo', sep=',')
    
connection.commit()    

In [7]:
# Create a new Role gastudent
cursor.execute("""CREATE ROLE gastudent WITH LOGIN PASSWORD 'gastudent'""")
#Grant different permissions for a new role
cursor.execute("""GRANT CONNECT ON DATABASE titanic TO gastudent""")
cursor.execute("""GRANT SELECT, UPDATE, INSERT ON passengers TO gastudent""")
cursor.execute("""GRANT SELECT, UPDATE, INSERT ON survival TO gastudent""")
cursor.execute("""GRANT SELECT, UPDATE, INSERT ON tripInfo TO gastudent""")

connection.commit()

In [ ]:
# simple query to interact with a remote DB.

sql = """
SELECT pclass, AVG(survived)
FROM (SELECT t.pclass, s.survived
FROM tripInfo t
INNER JOIN survival s ON t.id = s.id
) AS x
GROUP BY pclass
"""

pd.read_sql(sql, con=connection)